In [1]:
import numpy as np
import torch
from torch import optim

In [8]:
x_train = torch.FloatTensor([[73, 80, 75],
                             [93, 88, 93],
                             [89, 91, 90],
                             [96, 98, 100],
                             [73, 66, 70]])
y_train = torch.FloatTensor([[152], [185], [180], [196], [142]])

W = torch.zeros(3, requires_grad=True) # 1*3 matrix지만, 5*3 @ 1*3 할 때는 자동으로 5*3 @ 3*1 로 간주하나 봄
# W = torch.zeros((3, 1), requires_grad=True) # 이걸로 할 수도 있음!
b = torch.zeros(1, requires_grad=True)

hypothesis = x_train @ W + b
cost = torch.mean((hypothesis - y_train)**2)

# optimizer 정의
optimizer = optim.SGD([W, b], lr=1e-5)

# optimizer 사용 및 step
optimizer.zero_grad()
cost.backward()
optimizer.step()

In [20]:
# Full code with epochs
x_train = torch.FloatTensor([[73, 80, 75],
                             [93, 88, 93],
                             [89, 91, 90],
                             [96, 98, 100],
                             [73, 66, 70]])
y_train = torch.FloatTensor([[152], [185], [180], [196], [142]])

W = torch.zeros((3, 1), requires_grad=True)
b = torch.zeros(1, requires_grad=True)

optimizer = optim.SGD([W, b], lr=1e-5)

nb_epochs = 20
for epoch in range(1, nb_epochs+1):
    hypothesis = x_train @ W + b
    cost = torch.mean((hypothesis - y_train)**2)

    print(f"ep:{epoch} | W:{W.squeeze()} | b:{b.item():.3f}")

    optimizer.zero_grad()
    cost.backward()
    optimizer.step()

ep:1 | W:tensor([0., 0., 0.], grad_fn=<SqueezeBackward0>) | b:0.000
ep:2 | W:tensor([0.2940, 0.2936, 0.2974], grad_fn=<SqueezeBackward0>) | b:0.003
ep:3 | W:tensor([0.4586, 0.4579, 0.4639], grad_fn=<SqueezeBackward0>) | b:0.005
ep:4 | W:tensor([0.5508, 0.5499, 0.5571], grad_fn=<SqueezeBackward0>) | b:0.006
ep:5 | W:tensor([0.6025, 0.6014, 0.6093], grad_fn=<SqueezeBackward0>) | b:0.007
ep:6 | W:tensor([0.6314, 0.6302, 0.6385], grad_fn=<SqueezeBackward0>) | b:0.007
ep:7 | W:tensor([0.6476, 0.6463, 0.6549], grad_fn=<SqueezeBackward0>) | b:0.008
ep:8 | W:tensor([0.6568, 0.6553, 0.6640], grad_fn=<SqueezeBackward0>) | b:0.008
ep:9 | W:tensor([0.6619, 0.6603, 0.6692], grad_fn=<SqueezeBackward0>) | b:0.008
ep:10 | W:tensor([0.6648, 0.6630, 0.6720], grad_fn=<SqueezeBackward0>) | b:0.008
ep:11 | W:tensor([0.6664, 0.6646, 0.6737], grad_fn=<SqueezeBackward0>) | b:0.008
ep:12 | W:tensor([0.6674, 0.6654, 0.6746], grad_fn=<SqueezeBackward0>) | b:0.008
ep:13 | W:tensor([0.6679, 0.6658, 0.6751], grad_f

In [29]:
# W, b를 매번 정의해서 사용하면 비효율적, nn.module이라는 편리한 모듈을 제공하고 있다.
# Full code with nn.module
import torch
from torch import optim
import torch.nn as nn
import torch.nn.functional as F

x_train = torch.FloatTensor([[73, 80, 75],
                             [93, 88, 93],
                             [89, 91, 90],
                             [96, 98, 100],
                             [73, 66, 70]])
y_train = torch.FloatTensor([[152], [185], [180], [196], [142]])

class MultivariateLinearRegressionModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear = nn.Linear(3, 1)

    def forward(self, x):
        x = self.linear(x)
        return x

model = MultivariateLinearRegressionModel() # 직접 module을 만들어야 해?
optimizer = optim.SGD(model.parameters(), lr=1e-5)
nb_epochs = 20
for epoch in range(1, nb_epochs+1):

    pred = model(x_train)
    cost = F.mse_loss(pred, y_train) # mse loss function 사용

    print(f"ep:{epoch} | pred:{pred.squeeze().detach()} | cost:{cost:.6f}")

    optimizer.zero_grad()
    cost.backward()
    optimizer.step()

ep:1 | pred:tensor([29.9918, 37.8088, 36.2477, 40.2650, 28.6965]) | cost:18861.404297
ep:2 | pred:tensor([ 83.6247, 102.2720,  99.7642, 109.4329,  77.8659]) | cost:5912.791016
ep:3 | pred:tensor([113.6517, 138.3626, 135.3247, 148.1573, 105.3941]) | cost:1854.087158
ep:4 | pred:tensor([130.4626, 158.5685, 155.2337, 169.8377, 120.8062]) | cost:581.899292
ep:5 | pred:tensor([139.8743, 169.8811, 166.3800, 181.9757, 129.4350]) | cost:183.135651
ep:6 | pred:tensor([145.1434, 176.2147, 172.6203, 188.7713, 134.2661]) | cost:58.144054
ep:7 | pred:tensor([148.0934, 179.7608, 176.1141, 192.5759, 136.9709]) | cost:18.965687
ep:8 | pred:tensor([149.7448, 181.7461, 178.0701, 194.7059, 138.4853]) | cost:6.685191
ep:9 | pred:tensor([150.6693, 182.8577, 179.1651, 195.8984, 139.3333]) | cost:2.835761
ep:10 | pred:tensor([151.1867, 183.4801, 179.7782, 196.5660, 139.8082]) | cost:1.628994
ep:11 | pred:tensor([151.4763, 183.8287, 180.1213, 196.9397, 140.0742]) | cost:1.250555
ep:12 | pred:tensor([151.6384,